# Lab 3
Group Members: (Andrew Quijano, afq2003), (Isaiah Genis | ig596), (Jamie Leach | jhl925), (Yevhen Pankevych | yp2525)

## Problem 1
Answer Following Questions:  
a) Download the product sales data from the below link:  
https://drive.google.com/file/d/19Xu297MPLVFTR2VuvDLUPRi3o36ijqW6/view?usp=sharing  
Using Matplotlib library, plot the sales of each product for whole year (Y axis- Sales and X axis- Month). Use Dashed line for plot and different colors for each product.

b) Using Math library calculate the following:
- Ceiling of 435/6.
- Floor of 2667/8.
- Factorial of 25.
- Sum of the elements of array:
- [0.2, 0.33, 0.12, 0.57, 341.1, 23.77, 11.919, 9.1, 10.4, 4.4]

c) Use time library to find what time following tuple represents:  
t = (2007, 8, 10, 14, 20, 6, 7, 258, 0)  

d) What is socket library in Python? Why is it used? Give one brief example.  

e) What is multithreading?  
Using Threading library, write a Python program to calculate the factorial of number 21 and square root of number 91.  

In [21]:
# Question 1A
import matplotlib as plt

In [5]:
# Question 1B
import math

math.ceil(435/6)
math.floor(2667/8)
math.factorial(25)
math.fsum([0.2, 0.33, 0.12, 0.57, 341.1, 23.77, 11.919, 9.1, 10.4, 4.4])


401.90900000000005

In [ ]:
# Question 1C
import time
t = (2007, 8, 10, 14, 20, 6, 7, 258, 0) 

### Question 1D
The socket library is so your python program can communicate over the network. One example of how it would be used is to create a basic web server.

In [ ]:
# Question 1E

### Question 2
Please answer the following:  
a) Come up with three differences between symmetric and asymmetric encryption. Which one do you think is better? Explain why you think one is better than the other in a few sentences.

Differences:
- a
- b
- c



b) Is AES-256 symmetric or asymmetric?
AES-256 is a symmetric encryption system

### Question 3
Pydantic is a helpful library for data parsing and validation. It coerces input types to the declared type (using type hints), accumulates all the errors using ValidationError & it’s also well documented, making it easily discoverable.

a) Go over the documentation and try understanding how to use this library's methods.

b) Go over pydantic.py, which has classes written in pydantic, and a write up of what each class does and how pydantic is used.

### Question 4
Cerberus provides powerful yet simple and lightweight data validation functionality out of the box and is designed to be easily extensible, allowing for custom validation. It’s very similar to Pydantic and you go over the documentations here. It’s like Pydantic with slight variations.

a) Go over this article and modify the code in the given article to add extra validations to the given schema. Mind you that you can add extra key value pairs to the dictionary itself and then add validations for those.

b) Write your modifications.

In [70]:
from dataclasses import dataclass
import datetime


@dataclass
class Customer:
    id: int
    name: str
    accounts: list
    contact_details: dict
    role: str
    last_activity_date: datetime.date = datetime.date.today()
    access_right: str = ""
    client_card_type: str = ""
    

In [71]:
schema = {
  "id": {
    "type": "integer",
    "min": 0,
    "max": 100,
    "required": True
  },
  "name": {
    "type": "string",
    "minlength": 2,
    "maxlength": 255,
    "required": True
  },
  "accounts": {
    "type": "list",
    "schema": {
      "type": "integer",
      "min": 0,
      "max": 100
    }
  },
  "contact_details": {
    "type": "dict",
    "require_all": True,  # let every field be required
    "schema": {
      "phone": {
        "type": "string",
        "minlength": 11,
        "maxlength": 11,
        "regex": "^0[0-9]{10}$"
      },
      "email": {
        "type": "string",
        "minlength": 2,
        "maxlength": 255,
        "required": True,
        "regex": "^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$"
      }
    }
  },
  "role": {  # add field role only with 2 allowed values
    "type": "string", 
    "allowed": ['client', 'supplier']
  },
  "access_right": {  # add field access_right which can be set only if role == supplier
      "type": "string",
      "excludes": 'client_card_type',  # no client_card_type field if this field if present
      "empty": False,  # and it can`t be empty
      'dependencies': {
        "role": ['supplier'],
      },
      "anyof": [{"minlength": 2, "maxlength": 5}, {"minlength": 10, "maxlength": 15}]  # 2-5 or 10-15 characters
  },
  "client_card_type": {  # add field access_right which can be set only if role == client
    "type": "string",
    "excludes": 'access_right',  # no access_right field if this field if present
    "empty": False,  # and it can`t be empty
    "forbidden": ['root', 'admin'],  # no such values allowed for the field
    'dependencies': {
      "role": ['client'],
    }
  },
  "last_activity_date": {  # and create date field
    "type": "date",
  }
}


In [72]:
import json
import os

from cerberus import Validator


class Api:
    def create_customer(self, payload: dict) -> Customer:
        """
        validates payload and - only if valid - instantiates a Customer
        :param payload: payload containing Customer details
        :return: newly instantiated Customer if payload is valid
        """
        self.validate_payload(payload)

        return Customer(**payload)

    @staticmethod
    def validate_payload(payload: dict):
        """
        validate payload against a Cerberus schema
        :param payload: payload to be validated
        :return: nothing
        """
        validator = Validator()
        validator.validate(payload, schema)

        if validator.errors:
            print(f"Validation failed: {validator.errors}")
        else:
            print("OK")


In [81]:
# and tests

test_entries = [
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, "49"],
        "contact_details": {
          "phone": "01234567891",
        },
        "role": "student",
        "access_right": "admin",
        "client_card_type": "root",
    },
    
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, 49],
        "contact_details": {
          "phone": "01234567891",
          "email": "example@example.com"
        },
        "role": "client",
        "client_card_type": "premium",
    },
    
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, 49],
        "contact_details": {
          "phone": "01234567891",
          "email": "example@example.com"
        },
        "role": "client",
        "access_right": "admin",
    },
    
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, 49],
        "contact_details": {
          "phone": "01234567891",
          "email": "example@example.com"
        },
        "role": "supplier",
        "access_right": "admin",
    },
    
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, 49],
        "contact_details": {
          "phone": "01234567891",
          "email": "example@example.com"
        },
        "role": "supplier",
        "access_right": "myadmin",
    },
    
    {
        "id": 20,
        "name": "Hello",
        "accounts": [30, 17, 49],
        "contact_details": {
          "phone": "01234567891",
          "email": "example@example.com"
        },
        "role": "supplier",
        "access_right": "systemadmin",
        "last_activity_date": datetime.datetime.strptime('24.08.2021', "%d.%m.%Y").date()
    },
]

for entry in test_entries:
    print(f"Running: {entry}")
    Api().create_customer(entry)
    print()

print()

Running: {'id': 20, 'name': 'Hello', 'accounts': [30, 17, '49'], 'contact_details': {'phone': '01234567891'}, 'role': 'student', 'access_right': 'admin', 'client_card_type': 'root'}
Validation failed: {'access_right': ["depends on these values: {'role': ['supplier']}", "'client_card_type' must not be present with 'access_right'"], 'accounts': [{2: ['must be of integer type']}], 'client_card_type': ["depends on these values: {'role': ['client']}", "'access_right' must not be present with 'client_card_type'", 'unallowed value root'], 'contact_details': [{'email': ['required field']}], 'role': ['unallowed value student']}

Running: {'id': 20, 'name': 'Hello', 'accounts': [30, 17, 49], 'contact_details': {'phone': '01234567891', 'email': 'example@example.com'}, 'role': 'client', 'client_card_type': 'premium'}
OK

Running: {'id': 20, 'name': 'Hello', 'accounts': [30, 17, 49], 'contact_details': {'phone': '01234567891', 'email': 'example@example.com'}, 'role': 'client', 'access_right': 'admi